# 데이터 로딩

- glob, read_table 테스트


In [11]:
import glob, os
import pandas as pd

path = 'topics'
all_files = glob.glob(os.path.join(path, '*.data'))

# print(type(all_files))
# print(all_files)

df = pd.read_table(all_files[0], index_col=None, header =0,encoding='latin1')
print(df.shape)
display(df.head())
print(df.columns)

(66, 1)


,", and is very, very accurate ."
0,"but for the most part, we find that the Garmi..."
1,This function is not accurate if you don't le...
2,It provides immediate alternatives if the rou...
3,"I've used other GPS units, as well as GPS bui..."
4,It got me from point A to point B with 100% a...


Index([', and is very, very accurate .'], dtype='object')


In [13]:
pd.set_option('display.max_colwidth',700)

In [14]:
import glob, os
import pandas as pd

path = 'topics'
all_files = glob.glob(os.path.join(path, '*.data'))
filename_list = []
opinion_text = []

for file in all_files:
    df = pd.read_table(file, header = 0, encoding='latin1')
    filename = file.split('\\')[-1]
    filename = filename.split('.')[0]

    filename_list.append(filename)
    opinion_text.append(df.to_string())

document_df = pd.DataFrame({'filename':filename_list,'opinion':opinion_text})
document_df.head()


,filename,opinion
0,accuracy_garmin_nuvi_255W_gps,", and is very, very accurate .\n0 but for the most part, we find that the Garmin software provides accurate directions, whereever we intend to go .\n1 This functi..."
1,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and very comfortable beds, a great shower and very clean bathrooms .\n0 The second room was smaller, with a very inconvenient bathroom layout, but at least it was quieter and we were able to sleep .\n1 ..."
2,battery-life_amazon_kindle,"After I plugged it in to my USB hub on my computer to charge the battery the charging cord design is very clever !\n0 After you have paged tru a 500, page book one, page, at, a, time to get from Chapter 2 to Chapter 15, see how excited you are about a low battery and all the time it took to get there !\n1 ..."
3,battery-life_ipod_nano_8gb,short battery life I moved up from an 8gb .\n0 I love this ipod except for the battery life .\n1 ...
4,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh Li, ion Battery , and a 1 .\n0 Not to mention that as of now..."


# TfidfVectorizer의 tokenizer 인자로 사용될 lemmatization 어근 변환 함수 선언

- translate() 함수 
    - 문자열 내에 특정 문자를 다른 문자로 일괄 변경하는 함수
    - 매개변수로 변경하고자 하는 정보를 딕셔너리로 전달한다.

In [19]:
# maketrans : 원래 문자와 바꿀 문자에 대한 맵핑정보(딕셔너리)를 만든다.
# 원본 문자열과 바꿀 문자열의 길이가 같아야 한다.
# {원본 문자 유니코드: 변경 문자 유니코드}

trans = str.maketrans('[],','   ')
print(trans)
text = '[고양이,강아지,다람쥐,토끼]'
text = text.translate(trans)

print(text)

{91: 32, 93: 32, 44: 32}
 고양이 강아지 다람쥐 토끼 
